In [1]:
import warnings
warnings.filterwarnings("ignore")
import intake
from ncar_jobqueue import NCARCluster
import xarray as xr 
import dask
import yaml 
from pathlib import Path
import os

## Spin up Cluster

In [2]:
cluster = NCARCluster(memory="50GB")
cluster

In [5]:
cluster.adapt(minimum=0, maximum=36)

## Connect to intake-esm-database

In [6]:
#col = intake.open_esm_metadatastore(collection_input_definition="CESM1-LE", overwrite_existing=True)
col = intake.open_esm_metadatastore(collection_name="CESM1-LE")

In [7]:
col.df.head()

,resource,resource_type,direct_access,experiment,case,component,stream,variable,date_range,member_id,file_fullpath,file_basename,file_dirname,ctrl_branch_year,year_offset,sequence_order,has_ocean_bgc,grid
0,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,f.e11.F1850C5CN.f09_f09.001,atm,cam.h1,T200,16000101-16991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,f.e11.F1850C5CN.f09_f09.001.cam.h1.T200.160001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,NaN,0,False,NaN
1,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,f.e11.F1850C5CN.f09_f09.001,atm,cam.h1,T200,24000101-24991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,f.e11.F1850C5CN.f09_f09.001.cam.h1.T200.240001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,NaN,0,False,NaN
2,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,f.e11.F1850C5CN.f09_f09.001,atm,cam.h1,T200,11000101-11991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,f.e11.F1850C5CN.f09_f09.001.cam.h1.T200.110001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,NaN,0,False,NaN
3,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,f.e11.F1850C5CN.f09_f09.001,atm,cam.h1,T200,22000101-22991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,f.e11.F1850C5CN.f09_f09.001.cam.h1.T200.220001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,NaN,0,False,NaN
4,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,f.e11.F1850C5CN.f09_f09.001,atm,cam.h1,T200,25000101-26001231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,f.e11.F1850C5CN.f09_f09.001.cam.h1.T200.250001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,NaN,0,False,NaN


In [73]:
col.search(experiment='CTRL')

name: CESM1-LE_9b1446ae-2c21-4b35-908f-c329e3d4d9b7
container: xarray
plugin: ['cesm']
description: Catalog entry from CESM1-LE collection
direct_access: True
user_parameters: []
metadata: 
args: 
  collection_name: CESM1-LE
  query: 
    experiment: CTRL
    resource: None
    resource_type: None
    direct_access: None
    case: None
    component: None
    stream: None
    variable: None
    date_range: None
    member_id: None
    file_fullpath: None
    file_basename: None
    file_dirname: None
    ctrl_branch_year: None
    year_offset: None
    sequence_order: None
    has_ocean_bgc: None
    grid: None

In [8]:
def collection_summary(col):
    df = col.df
    print("There are:\n")
    print(f"\t1) {df.experiment.nunique()} experiments: \n\n\t\t{df.experiment.unique()}\n")
    print(f"\t2) {df.case.nunique()} cases: \n\n\t\t{df.case.unique()[:5]} etc...\n")
    print(f"\t3) {df.component.nunique()} components: \n\n\t\t{df.component.unique()}\n")
    print(f"\t4) {df.stream.nunique()} streams: \n\n\t\t{df.stream.unique()}\n")
    print(f"\t5) {df.variable.nunique()} variables: \n\n\t\t{df.variable.unique()[:10]} etc...\n")
    
collection_summary(col)

There are:

	1) 3 experiments: 

		['CTRL' '20C' 'RCP85']

	2) 93 cases: 

		['f.e11.F1850C5CN.f09_f09.001' 'e.e11.E1850C5CN.f09_g16.001'
 'b.e11.B1850C5CN.f09_g16.005' 'b.e11.B20TRC5CNBDRD.f09_g16.OIC.010'
 'b.e11.B20TRC5CNBDRD.f09_g16.OIC.009'] etc...

	3) 5 components: 

		['atm' 'lnd' 'rof' 'ice' 'ocn']

	4) 10 streams: 

		['cam.h1' 'cam.h0' 'clm2.h1' 'clm2.h0' 'rtm.h1' 'rtm.h0' 'cice.h1'
 'cice.h' 'pop.h.nday1' 'pop.h']

	5) 985 variables: 

		['T200' 'Q200' 'FLNSC' 'Q500' 'PRECL' 'QBOT' 'TREFHTMN' 'FSNTOA' 'TAUX'
 'TAUY'] etc...



In [31]:
def print_query_info(query):
    cat = col.search(**query)
    query_results = cat.query_results
    variables = sorted(query_results.variable.unique().tolist())
    files = sorted(query_results.file_fullpath.unique().tolist())
    date_ranges = sorted(query_results.date_range.unique())
    cases = sorted(query_results.case.unique())
    
    print(f"This query returned:\n\n{len(variables)} unique variables: {variables}\n\n")
    print(f"Number of files: {len(files)}\n\n")
    print(f"Date ranges: {date_ranges}\n\n")
    print(f"{len(cases)} cases: {cases}\n\n")

## Variables to include in AWS

### Atmosphere, monthly, 3D

In [34]:
atmos_mon_3d = ["T", "U", "V", "Q", "Z3"]
query = dict(variable=atmos_mon_3d, component="atm", stream="cam.h0")
print_query_info(query)

This query returned:

5 unique variables: ['Q', 'T', 'U', 'V', 'Z3']


Number of files: 880


Date ranges: ['000101-009912', '010001-019912', '010101-019912', '020001-029912', '030001-039912', '040001-049912', '050001-059912', '060001-069912', '070001-079912', '080001-089912', '090001-099912', '090001-100112', '100001-109912', '110001-119912', '120001-129912', '130001-139912', '140001-149912', '150001-159912', '160001-169912', '170001-179912', '180001-189912', '185001-199912', '185001-200512', '190001-199912', '192001-199912', '192001-200512', '200001-209912', '200601-208012', '200601-210012', '208101-210012', '210001-219912', '210001-220012', '220001-229912', '230001-239912', '240001-249912', '250001-260012']


93 cases: ['b.e11.B1850C5CN.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.001', 'b.e11.B20TRC5CNBDRD.f09_g16.002', 'b.e11.B20TRC5CNBDRD.f09_g16.003', 'b.e11.B20TRC5CNBDRD.f09_g16.004', 'b.e11.B20TRC5CNBDRD.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.006', 'b.e11.B20TRC5CNBDRD.f09_g

**NOTE:**
    
- There are **880** files. The spreadsheet mentions **1010** files. 

### Atmosphere, Monthly, Surface

In [36]:
atmos_mon_surface = ["FLNS", "FLNSC", "FLUT", "FSNS", "FSNSC", "FSNTOA", "ICEFRAC", 
                     "LHFLX", "PRECC", "PRECL", "PRECSC", "PRECSL", "PSL", "SHFLX", 
                     "TMQ", "TREFHT", "TREFHTMN", "TREFHTMX", "TS"]

In [37]:
query = dict(variable=atmos_mon_surface, component="atm", stream="cam.h0")
print_query_info(query)

This query returned:

19 unique variables: ['FLNS', 'FLNSC', 'FLUT', 'FSNS', 'FSNSC', 'FSNTOA', 'ICEFRAC', 'LHFLX', 'PRECC', 'PRECL', 'PRECSC', 'PRECSL', 'PSL', 'SHFLX', 'TMQ', 'TREFHT', 'TREFHTMN', 'TREFHTMX', 'TS']


Number of files: 3345


Date ranges: ['000101-009912', '010001-019912', '010101-019912', '020001-029912', '030001-039912', '040001-049912', '050001-059912', '060001-069912', '070001-079912', '080001-089912', '090001-099912', '090001-100112', '100001-109912', '110001-119912', '120001-129912', '130001-139912', '140001-149912', '150001-159912', '160001-169912', '170001-179912', '180001-189912', '185001-199912', '185001-200512', '190001-199912', '192001-199912', '192001-200512', '200001-209912', '200601-208012', '200601-210012', '208101-210012', '210001-219912', '210001-220012', '220001-229912', '230001-239912', '240001-249912', '250001-260012']


93 cases: ['b.e11.B1850C5CN.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.001', 'b.e11.B20TRC5CNBDRD.f09_g16.002', 'b.e11.B20TRC5CNB

**NOTE:**
    
- There are **3345** files. The spreadsheet mentions **3839** files. 

### Land, Monthly

In [38]:
land_vars = ["FSNO", "H2OSNO", "QRUNOFF", "RAIN", "SNOW", "SOILLIQ", "SOILWATER_10CM"]

In [39]:
query = dict(variable=land_vars, component="lnd", stream="clm2.h0")
print_query_info(query)

This query returned:

7 unique variables: ['FSNO', 'H2OSNO', 'QRUNOFF', 'RAIN', 'SNOW', 'SOILLIQ', 'SOILWATER_10CM']


Number of files: 1232


Date ranges: ['000101-009912', '010001-019912', '010101-019912', '020001-029912', '030001-039912', '040001-049912', '050001-059912', '060001-069912', '070001-079912', '080001-089912', '090001-099912', '090001-100112', '100001-109912', '110001-119912', '120001-129912', '130001-139912', '140001-149912', '150001-159912', '160001-169912', '170001-179912', '180001-189912', '185001-199912', '185001-200512', '190001-199912', '192001-199912', '192001-200512', '200001-209912', '200601-208012', '200601-210012', '208101-210012', '210001-219912', '210001-220012', '220001-229912', '230001-239912', '240001-249912', '250001-260012']


93 cases: ['b.e11.B1850C5CN.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.001', 'b.e11.B20TRC5CNBDRD.f09_g16.002', 'b.e11.B20TRC5CNBDRD.f09_g16.003', 'b.e11.B20TRC5CNBDRD.f09_g16.004', 'b.e11.B20TRC5CNBDRD.f09_g16.005', 'b.e11.B20TR

### Land, Daily

In [40]:
query = dict(variable=land_vars, component="lnd", stream="clm2.h1")
print_query_info(query)

This query returned:

6 unique variables: ['FSNO', 'H2OSNO', 'QRUNOFF', 'RAIN', 'SNOW', 'SOILWATER_10CM']


Number of files: 1056


Date ranges: ['00010101-00991231', '01000101-01991231', '01010101-01991231', '02000101-02991231', '03000101-03991231', '04000101-04991231', '04030101-04991231', '05000101-05991231', '06000101-06991231', '07000101-07991231', '08000101-08991231', '09000101-09991231', '09000101-10011231', '10000101-10991231', '11000101-11991231', '12000101-12991231', '13000101-13991231', '14000101-14991231', '15000101-15991231', '16000101-16991231', '17000101-17991231', '18000101-18991231', '18500101-19991231', '18500101-20051231', '19000101-19991231', '19200101-19991231', '19200101-20051231', '20000101-20991231', '20060101-20801231', '20060101-21001231', '20810101-21001231', '21000101-21991231', '21000101-22001231', '22000101-22991231', '23000101-23991231', '24000101-24991231', '25000101-26001231']


93 cases: ['b.e11.B1850C5CN.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.001'

**NOTE:**
    
- There are **1056+1232=2288** files. The spreadsheet mentions **1414** files. 

### Ocean, Monthly, Surface

In [41]:
ocn_surface_vars = ["SST", "SSH"]

In [42]:
query = dict(variable=ocn_surface_vars, component="ocn", stream="pop.h")
print_query_info(query)

This query returned:

2 unique variables: ['SSH', 'SST']


Number of files: 282


Date ranges: ['040001-049912', '050001-059912', '060001-069912', '070001-079912', '080001-089912', '090001-099912', '100001-109912', '110001-119912', '120001-129912', '130001-139912', '140001-149912', '150001-159912', '160001-169912', '170001-179912', '180001-189912', '185001-199912', '185001-200512', '190001-199912', '192001-199912', '192001-200512', '200001-209912', '200601-208012', '200601-210012', '208101-210012', '210001-220012']


91 cases: ['b.e11.B1850C5CN.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.001', 'b.e11.B20TRC5CNBDRD.f09_g16.002', 'b.e11.B20TRC5CNBDRD.f09_g16.003', 'b.e11.B20TRC5CNBDRD.f09_g16.004', 'b.e11.B20TRC5CNBDRD.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.006', 'b.e11.B20TRC5CNBDRD.f09_g16.007', 'b.e11.B20TRC5CNBDRD.f09_g16.008', 'b.e11.B20TRC5CNBDRD.f09_g16.009', 'b.e11.B20TRC5CNBDRD.f09_g16.010', 'b.e11.B20TRC5CNBDRD.f09_g16.011', 'b.e11.B20TRC5CNBDRD.f09_g16.012', 'b.e11.B20TRC5C

### Ocean, Monthly, 3D

In [44]:
ocn_3d_vars = ["SALT"]
query = dict(variable=ocn_3d_vars, component="ocn", stream="pop.h")
print_query_info(query)

This query returned:

1 unique variables: ['SALT']


Number of files: 141


Date ranges: ['040001-049912', '050001-059912', '060001-069912', '070001-079912', '080001-089912', '090001-099912', '100001-109912', '110001-119912', '120001-129912', '130001-139912', '140001-149912', '150001-159912', '160001-169912', '170001-179912', '180001-189912', '185001-199912', '185001-200512', '190001-199912', '192001-199912', '192001-200512', '200001-209912', '200601-208012', '200601-210012', '208101-210012', '210001-220012']


91 cases: ['b.e11.B1850C5CN.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.001', 'b.e11.B20TRC5CNBDRD.f09_g16.002', 'b.e11.B20TRC5CNBDRD.f09_g16.003', 'b.e11.B20TRC5CNBDRD.f09_g16.004', 'b.e11.B20TRC5CNBDRD.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.006', 'b.e11.B20TRC5CNBDRD.f09_g16.007', 'b.e11.B20TRC5CNBDRD.f09_g16.008', 'b.e11.B20TRC5CNBDRD.f09_g16.009', 'b.e11.B20TRC5CNBDRD.f09_g16.010', 'b.e11.B20TRC5CNBDRD.f09_g16.011', 'b.e11.B20TRC5CNBDRD.f09_g16.012', 'b.e11.B20TRC5CNBDRD.

**NOTE:**
    
- There are **282+141=423** files. The spreadsheet mentions **438** files. 

### Ice, Monthly

In [46]:
ice_mon_vars = ["aice_nh", "aice_sh", "hi_nh", "hi_sh"]
query = dict(variable=ice_mon_vars, component="ice", stream="cice.h")
print_query_info(query)

This query returned:

4 unique variables: ['aice_nh', 'aice_sh', 'hi_nh', 'hi_sh']


Number of files: 600


Date ranges: ['010101-019912', '020001-029912', '030001-039912', '040001-049912', '050001-059912', '060001-069912', '070001-079912', '080001-089912', '090001-099912', '090001-100112', '100001-109912', '110001-119912', '120001-129912', '130001-139912', '140001-149912', '150001-159912', '160001-169912', '170001-179912', '180001-189912', '185001-199912', '185001-200512', '190001-199912', '192001-199912', '192001-200512', '200001-209912', '200601-208012', '200601-210012', '208101-210012', '210001-220012']


92 cases: ['b.e11.B1850C5CN.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.001', 'b.e11.B20TRC5CNBDRD.f09_g16.002', 'b.e11.B20TRC5CNBDRD.f09_g16.003', 'b.e11.B20TRC5CNBDRD.f09_g16.004', 'b.e11.B20TRC5CNBDRD.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.006', 'b.e11.B20TRC5CNBDRD.f09_g16.007', 'b.e11.B20TRC5CNBDRD.f09_g16.008', 'b.e11.B20TRC5CNBDRD.f09_g16.009', 'b.e11.B20TRC5CNBDRD.f09_g

**NOTE:**
    
- There are **600** files. The spreadsheet mentions **620** files. 

I would still urge that we focus on 1920-2100, concatenate 20C & RCP8.5, combine ATM+LND and OCN+ICE if possible, and ignore the various ancillary runs, ensemble members #106-107, etc.

For the monthly data, I think these regular expressions will list only, and all, the desired files:
Atmosphere & Land:
ls */proc/tseries/monthly/{T,U,V,Q,Z3,FLNS,FLNSC,FLUT,FSNS,FSNSC,FSNTOA,ICEFRAC,LHFLX,PRECC,PRECL,PRECSC,PRECSL,PSL,SHFLX,TMQ,TREFHT,TREFHTMN,TREFHTMX,TS,FSNO,H2OSNO,QRUNOFF,RAIN,SNOW,SOILLIQ,SOILWATER_10CM}/{b.e11.B20TRC5CNBDRD.f09_g16.{0??,10[1-5]}.*.1920*.nc,b.e11.BRCP85C5CNBDRD.f09_g16.*.nc}
Ocean & Ice:
ls */proc/tseries/monthly/{SSH,SST,SALT,aice,hi}/{b.e11.B20TRC5CNBDRD.f09_g16.{0??,10[1-5]}.*.1920*.nc,b.e11.BRCP85C5CNBDRD.f09_g16.*.nc}

In [69]:
import numpy as np

In [72]:
np.object == col.df.case.dtype

True

In [63]:
atm_lnd_variables = list(set(atmos_mon_3d + atmos_mon_surface + land_vars))
members = list(range(1, 36)) + list(range(101, 106))

cases = ['b.e11.B20TRC5CNBDRD.f09_g16.001', 'b.e11.B20TRC5CNBDRD.f09_g16.002', 'b.e11.B20TRC5CNBDRD.f09_g16.003', 
'b.e11.B20TRC5CNBDRD.f09_g16.004', 'b.e11.B20TRC5CNBDRD.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.006', 
'b.e11.B20TRC5CNBDRD.f09_g16.007', 'b.e11.B20TRC5CNBDRD.f09_g16.008', 'b.e11.B20TRC5CNBDRD.f09_g16.009', 
'b.e11.B20TRC5CNBDRD.f09_g16.010', 'b.e11.B20TRC5CNBDRD.f09_g16.011', 'b.e11.B20TRC5CNBDRD.f09_g16.012', 
'b.e11.B20TRC5CNBDRD.f09_g16.013', 'b.e11.B20TRC5CNBDRD.f09_g16.014', 'b.e11.B20TRC5CNBDRD.f09_g16.015', 
'b.e11.B20TRC5CNBDRD.f09_g16.016', 'b.e11.B20TRC5CNBDRD.f09_g16.017', 'b.e11.B20TRC5CNBDRD.f09_g16.018', 
'b.e11.B20TRC5CNBDRD.f09_g16.019', 'b.e11.B20TRC5CNBDRD.f09_g16.020', 'b.e11.B20TRC5CNBDRD.f09_g16.021', 
'b.e11.B20TRC5CNBDRD.f09_g16.022', 'b.e11.B20TRC5CNBDRD.f09_g16.023', 'b.e11.B20TRC5CNBDRD.f09_g16.024', 
'b.e11.B20TRC5CNBDRD.f09_g16.025', 'b.e11.B20TRC5CNBDRD.f09_g16.026', 'b.e11.B20TRC5CNBDRD.f09_g16.027', 
'b.e11.B20TRC5CNBDRD.f09_g16.028', 'b.e11.B20TRC5CNBDRD.f09_g16.029', 'b.e11.B20TRC5CNBDRD.f09_g16.030', 
'b.e11.B20TRC5CNBDRD.f09_g16.031', 'b.e11.B20TRC5CNBDRD.f09_g16.032', 'b.e11.B20TRC5CNBDRD.f09_g16.033', 
'b.e11.B20TRC5CNBDRD.f09_g16.034', 'b.e11.B20TRC5CNBDRD.f09_g16.035', 'b.e11.B20TRC5CNBDRD.f09_g16.101', 
'b.e11.B20TRC5CNBDRD.f09_g16.102', 'b.e11.B20TRC5CNBDRD.f09_g16.103', 'b.e11.B20TRC5CNBDRD.f09_g16.104', 
         'b.e11.B20TRC5CNBDRD.f09_g16.105']
query = dict(variable=atm_lnd_variables, component=["atm", "lnd"], experiment=["20C", "RCP85"], case=cases)
print_query_info(query)

This query returned:

31 unique variables: ['FLNS', 'FLNSC', 'FLUT', 'FSNO', 'FSNS', 'FSNSC', 'FSNTOA', 'H2OSNO', 'ICEFRAC', 'LHFLX', 'PRECC', 'PRECL', 'PRECSC', 'PRECSL', 'PSL', 'Q', 'QRUNOFF', 'RAIN', 'SHFLX', 'SNOW', 'SOILLIQ', 'SOILWATER_10CM', 'T', 'TMQ', 'TREFHT', 'TREFHTMN', 'TREFHTMX', 'TS', 'U', 'V', 'Z3']


Number of files: 2201


Date ranges: ['185001-200512', '18500101-20051231', '192001-200512', '19200101-20051231']


40 cases: ['b.e11.B20TRC5CNBDRD.f09_g16.001', 'b.e11.B20TRC5CNBDRD.f09_g16.002', 'b.e11.B20TRC5CNBDRD.f09_g16.003', 'b.e11.B20TRC5CNBDRD.f09_g16.004', 'b.e11.B20TRC5CNBDRD.f09_g16.005', 'b.e11.B20TRC5CNBDRD.f09_g16.006', 'b.e11.B20TRC5CNBDRD.f09_g16.007', 'b.e11.B20TRC5CNBDRD.f09_g16.008', 'b.e11.B20TRC5CNBDRD.f09_g16.009', 'b.e11.B20TRC5CNBDRD.f09_g16.010', 'b.e11.B20TRC5CNBDRD.f09_g16.011', 'b.e11.B20TRC5CNBDRD.f09_g16.012', 'b.e11.B20TRC5CNBDRD.f09_g16.013', 'b.e11.B20TRC5CNBDRD.f09_g16.014', 'b.e11.B20TRC5CNBDRD.f09_g16.015', 'b.e11.B20TRC5CNBDRD.f09_g16.